# Electronics / GPIO (General Purpose IO)

## Introduction

A **Breadboard** is a reusbale device with a number of holes into which you push wires or components to create circuits. Breadboards  allow you to create circuits without needing to solder all the components. The two rows of holes at the top and bottom are for power.
![Empty Breadboard](images/breadboard_empty.jpg)

**Voltage** is the difference in electrical energy between two points in a circuit. This difference allows current to flow in a circuit, like water pressure in pipes. Voltage is measured in volts (V).

An **LED** (Light-Emitting Diode) ligts up when electricty passes through it. A diode only allows current to pass in one direction and will only light up if you pass your current in the correct direction. LEDs have one short leg (cathode or negative) and one long leg (anode or positive).

A **resistor** resists or limits current in a circuit. An LED can be damaged by too much current and adding a resistor of the correct value will limit the amount of current and the LED will be protected. Resistance is measured in Ohms and the value of a resistor is shown by coloured bands that are read from left to right. You can read more about color code on [wikipedia](https://en.wikipedia.org/wiki/Electronic_color_code). 

A **current** is the rate at which electrical energy flows through a circuit and is measured in amperes (A) or amps. Smaller currents are measured in milliApms (mA). You can read more about circuits [here](http://www.allaboutcircuits.com/textbook/direct-current/#chpt-1).


## Setup instructions
* The Raspberry PI should be up and running, no need to shut it down.
* Insert the breakout board into the breadboard, being careful to line up the pins like on the picture below.
* Connect the header cable between the Raspberry PI and the breakout board with the correct orientation (look at the red part of the cable ribbon).

![Empty Breadboard](images/breadboard_empty.jpg)

![RPI and Breadboard](images/rpi_and_breadboard.jpg)

Through the rest of the lesson, keep the breadboard oriented in a way that the 
lettering on the breakout board is visible to you (i.e. you should see SDA, GND, 
etc).


## A simple circuit

- Connect an LED, a 330 Ohm resistor and two short wires to the positive + (red line) and negative - (blue line) rows as shown.

![Single LED](images/led_setup.jpg)


## A programmable circuit

- Connect the positive + wire to GPIO pin 17.

![Single LED](images/blinking_led_setup.jpg)

In [1]:
# run this code first, only need to run it once
from minecraft_utils import *

**Flash LED only once**. You should see the light turn on for one second.
* can you make it turn on for 5 seconds ?

In [3]:
LED = 17
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED, GPIO.OUT)  # setup for output

GPIO.output(LED, True)
time.sleep(1.0)

GPIO.cleanup()  # reset GPIO and disble circuitry

**Try it!** Make your LED flash forever
* Will this program work ? Try it
* If it doesn't work, fix it !

In [ ]:
LED = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED, GPIO.OUT)
seconds=1

def flash(t):
    GPIO.output(LED, True)  # LED on
    time.sleep(t)
    GPIO.output(LED, False) # LED off
    time.sleep(t)

try:
    while True:
        flash(seconds)  # how many seconds to flash?
finally:
    GPIO.cleanup()

**Try it!** 
Can you add more LEDs and make them flash?

In [ ]:
GPIO.setmode(GPIO.BCM)
LED_LIST = [17,18]  # add more!!

for LED in LED_LIST:
    GPIO.setup(LED, GPIO.OUT)

def flash(LED, t):
    GPIO.output(LED, True)  # LED on
    time.sleep(t)
    GPIO.output(LED, False) # LED off
    time.sleep(t)

try:
    while True:
        for LED in LED_LIST:
            flash(LED, 0.2)  # how many seconds to flash?
finally:
    GPIO.cleanup()

## The Intruder Alarm

Using the circuit you've created above, we will create an **Intruder Alarm**. It will tell us whether someone is inside our house, or our fence.

- Build a rectangular fence like on the screenshot below, or use a house that you've created previously.
- Previously, we're written some code to react based on your position compared to a wall. 
- The code will be similar, will need to figure out whether your current position is:
  - below the northern wall
  - above the southern wall
  - right of the leftmost wall
  - left of the rightmost wall
- Once the code is written, test it out by jumping in and out of your fence. The LED should light up when inside, and turn off when outside.

![LED Fence](screenshots/led_fence.png)

Solution: [fence_led.py](../scripts/fence_led.py)

- :bulb: Make the LED flash when the intruder is getting **close** to the house, but not yet inside the house.
- :bulb: If you have three LEDs, successive LEDs light up as you get closer, and then all together when you enter the house.

In [ ]:
import RPi.GPIO as GPIO
import mcpi.minecraft as minecraft
import time
mc = minecraft.Minecraft.create()


LED = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED, GPIO.OUT)  # setup for output

led_status = False

# you will have to figure out these numbers by walking along the edges of your fence
z_max = 11
z_min = 6
x_max = 61
x_min = 55


while True:
    time.sleep(0.25)
    pos = mc.player.getTilePos()
    inside_fence = False
    if pos.z > z_min and pos.z < z_max:
        if pos.x > x_min and pos.x < x_max:
            inside_fence = True
    if inside_fence != led_status:
        led_status = inside_fence
        GPIO.output(LED, led_status)
        

## Programmable Button

- Add a button to the circuit: the button has four pins and fits nicely accross the midway point of the breadboard.
- Connect the button to GPIO pin 4 (positive +) and GND (negative -) as shown.
- When the button is not pressed (and no connection is made) the wire that connects the GPIO will be at 3.3 volts and the GPIO pin will interpret this as signal '1' or True (try printing the value of `GPIO.input(BUTTON)`). When you press the button, the circuit between the left and right hand pins will be completed abnd the GPIO wire is 'pulled down' to 0 volts. Now the GPIO pin will interpret the signal as '0' or False.
 
![pushbutton](images/pushbutton.jpg)

In [2]:
BUTTON = 4
GPIO.setmode(GPIO.BCM)
GPIO.setup(BUTTON, GPIO.IN, pull_up_down=GPIO.PUD_UP)   # setup for input

try:
    while GPIO.input(BUTTON) == True:
        time.sleep(0.2)
      
    mc.postToChat('Button pressed!')
    
finally:
    GPIO.cleanup()  # reset GPIO and disable circuitry

## Magic button

Using the circuit above, you can program your button to do whatever you want, like blowing things up or building a house (or castle? try calling castle()) in minecraft.


In [ ]:
BUTTON = 4
GPIO.setmode(GPIO.BCM)
GPIO.setup(BUTTON, GPIO.IN, pull_up_down=GPIO.PUD_UP)   # setup for input

def doSomethingMagical():
    bomb()

try:
    while True:
        while GPIO.input(BUTTON) == True:
            time.sleep(0.2)
        mc.postToChat('Button pressed!')
        doSomethingMagical()
    
finally:
    GPIO.cleanup()  # reset GPIO and disable circuitry

Can we add some LEDs for countdown?

In [ ]:
LED_LIST = [18]  # add more!!
BUTTON = 4
GPIO.setmode(GPIO.BCM)
GPIO.setup(BUTTON, GPIO.IN, pull_up_down=GPIO.PUD_UP)   # setup for input

for LED in LED_LIST:
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(LED, GPIO.OUT)

def doSomethingMagical():
    bomb()

try:
    for LED in LED_LIST:
            GPIO.output(LED, True)
    while GPIO.input(BUTTON) == True:
        time.sleep(0.2)
    mc.postToChat('Button pressed!')
    for t in range(3):
        for LED in LED_LIST:
            GPIO.output(LED, False)
        time.sleep(1)
        for LED in LED_LIST:
            GPIO.output(LED, True)
        time.sleep(1)
    doSomethingMagical()
    
finally:
    GPIO.cleanup()  # reset GPIO and disable circuitry

## Smart home lighting

Using the flashing LED circuit you've created above, we will create an **Smart lighting for your house**. It will detect whether there's someone in the house and turn the light on and off automatically (isn't it nice?)

- Build a rectangular fence like on the screenshot below, or use a house that you've created previously.
- Previously, we're written some code to react based on your position compared to a wall. 
- The code will be similar, will need to figure out whether your current position is:
  - below the northern wall
  - above the southern wall
  - right of the leftmost wall
  - left of the rightmost wall
- Once the code is written, test it out by jumping in and out of your fence. The LED should light up when inside, and turn off when outside.

![LED Fence](screenshots/led_fence.png)


In [ ]:
LED = 17
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED, GPIO.OUT)

# the function below is not complete. It's your job to make it work !
def inside_house():
    return False

try:
    while True:
        if inside_house():
            GPIO.output(LED, True)  # LED on
        else:
            GPIO.output(LED, False)  # LED off
            
finally:
    GPIO.cleanup()

**Well done!** Now try apply what you just learnt and create something amazing!

In [ ]:
village()